# Lettura dei dati

In [0]:
import requests
from pyspark.sql.functions import isnan, when, count, isnull

In [0]:
local_path_popolazione_regioni = "dbfs:/FileStore/shared_uploads/gu.dichiara@studenti.unina.it/popolazione_regioni.csv"
local_path_dati_covid_province = "dbfs:/FileStore/shared_uploads/gu.dichiara@studenti.unina.it/dati_covid_province.csv"
local_path_dati_covid_regioni = "dbfs:/FileStore/shared_uploads/gu.dichiara@studenti.unina.it/dati_covid_regioni.csv"
local_path_dati_vaccini_somministrazioni = "dbfs:/FileStore/shared_uploads/gu.dichiara@studenti.unina.it/dati_vaccini_somministrazioni.csv"
local_path_dati_vaccini_consegne = "dbfs:/FileStore/shared_uploads/gu.dichiara@studenti.unina.it/dati_vaccini_consegne.csv"

# local_path_popolazione_regioni = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/popolazione_regioni.csv"
# local_path_dati_covid_province = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/dati_covid_province.csv"
# local_path_dati_covid_regioni = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/dati_covid_regioni.csv"
# local_path_dati_vaccini_somministrazioni = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/dati_vaccini_somministrazioni.csv"
# local_path_dati_vaccini_consegne = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/dati_vaccini_consegne.csv"

popolazione_regioni_URL = "https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-statistici-riferimento/popolazione-istat-regione-range.csv"
dati_covid_province_URL = "https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-province/dpc-covid19-ita-province.csv"
dati_covid_regioni_URL = "https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv"
dati_vaccini_somministrazioni_URL = "https://raw.githubusercontent.com/italia/covid19-opendata-vaccini/master/dati/somministrazioni-vaccini-latest.csv"
dati_vaccini_consegne_URL = "https://raw.githubusercontent.com/italia/covid19-opendata-vaccini/master/dati/consegne-vaccini-latest.csv"

dbutils.fs.put(local_path_popolazione_regioni, requests.get(popolazione_regioni_URL).text, True)
dbutils.fs.put(local_path_dati_covid_province, requests.get(dati_covid_province_URL).text, True)
dbutils.fs.put(local_path_dati_covid_regioni, requests.get(dati_covid_regioni_URL).text, True)
dbutils.fs.put(local_path_dati_vaccini_somministrazioni, requests.get(dati_vaccini_somministrazioni_URL).text, True)
dbutils.fs.put(local_path_dati_vaccini_consegne, requests.get(dati_vaccini_consegne_URL).text, True)


popolazione_regioni = spark.read.option("inferSchema", "true").option("header", "true").csv(local_path_popolazione_regioni)
dati_covid_province = spark.read.option("inferSchema", "true").option("header", "true").csv(local_path_dati_covid_province)
dati_covid_regioni = spark.read.option("inferSchema", "true").option("header", "true").csv(local_path_dati_covid_regioni)
dati_vaccini_somministrazioni = spark.read.option("inferSchema", "true").option("header", "true").csv(local_path_dati_vaccini_somministrazioni)
dati_vaccini_consegne = spark.read.option("inferSchema", "true").option("header", "true").csv(local_path_dati_vaccini_consegne)

Wrote 17300 bytes.
Wrote 5385839 bytes.
Wrote 1472383 bytes.
Wrote 3668020 bytes.
Wrote 54925 bytes.

# Preprocessing

## Popolazione Regioni

In [0]:
popolazione_regioni = popolazione_regioni.drop('codice_regione', 'latitudine_regione', 'longitudine_regione')

popolazione_regioni.show()

# popolazione_regioni.groupBy('denominazione_regione').sum('totale_generale').orderBy('sum(totale_generale)', ascending=False).show()

+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+
codice_nuts_1|descrizione_nuts_1|codice_nuts_2|denominazione_regione|sigla_regione|range_eta|totale_genere_maschile|totale_genere_femminile|totale_generale|
+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+
 ITF| Sud| ITF1| Abruzzo| ABR| 0-15| 87423| 82474| 169897|
 ITF| Sud| ITF1| Abruzzo| ABR| 16-19| 24402| 22349| 46751|
 ITF| Sud| ITF1| Abruzzo| ABR| 20-29| 67472| 62611| 130083|
 ITF| Sud| ITF1| Abruzzo| ABR| 30-39| 76472| 74241| 150713|
 ITF| Sud| ITF1| Abruzzo| ABR| 40-49| 94745| 95315| 190060|
 ITF| Sud| ITF1| Abruzzo| ABR| 50-59| 100048| 104109| 204157|
 ITF| Sud| ITF1| Abruzzo| ABR| 60-69| 80545| 87160| 167705|
 ITF| Sud| ITF1| Abruzzo| ABR| 70-79| 60810| 69762| 130572|
 ITF| Sud| ITF1| Abruzzo| ABR| 80-89| 34181| 50307| 84488|
 ITF| Sud| ITF1| Abruzzo| ABR| 90+| 5645| 13870| 19515|
 ITF| Sud| ITF5| Basilicata| BAS| 0-15| 36113| 33881| 69994|
 ITF| Sud| ITF5| Basilicata| BAS| 16-19| 11598| 10434| 22032|
 ITF| Sud| ITF5| Basilicata| BAS| 20-29| 32068| 29115| 61183|
 ITF| Sud| ITF5| Basilicata| BAS| 30-39| 33556| 31648| 65204|
 ITF| Sud| ITF5| Basilicata| BAS| 40-49| 39260| 39207| 78467|
 ITF| Sud| ITF5| Basilicata| BAS| 50-59| 43150| 44672| 87822|
 ITF| Sud| ITF5| Basilicata| BAS| 60-69| 35360| 37457| 72817|
 ITF| Sud| ITF5| Basilicata| BAS| 70-79| 24050| 27755| 51805|
 ITF| Sud| ITF5| Basilicata| BAS| 80-89| 14507| 21600| 36107|
 ITF| Sud| ITF5| Basilicata| BAS| 90+| 2488| 5335| 7823|
+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+
only showing top 20 rows

In [0]:
popolazione_regioni.select([count(when(isnull(c), c)).alias(c) for c in popolazione_regioni.columns]).show()

+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+
codice_nuts_1|descrizione_nuts_1|codice_nuts_2|denominazione_regione|sigla_regione|range_eta|totale_genere_maschile|totale_genere_femminile|totale_generale|
+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+
 0| 0| 0| 0| 0| 0| 0| 0| 0|
+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+

In [0]:
num_regioni = popolazione_regioni.groupBy('denominazione_regione') \
                .count() \
                .count()
print('Il numero di regioni è:', num_regioni)

popolazione_regioni.groupBy('descrizione_nuts_1', 'denominazione_regione', 'sigla_regione') \
                    .count() \
                    .drop('count') \
                    .orderBy('descrizione_nuts_1').show(num_regioni)

Il numero di regioni è: 21
+------------------+---------------------+-------------+
descrizione_nuts_1|denominazione_regione|sigla_regione|
+------------------+---------------------+-------------+
 Centro| Umbria| UMB|
 Centro| Toscana| TOS|
 Centro| Marche| MAR|
 Centro| Lazio| LAZ|
 Isole| Sardegna| SAR|
 Isole| Sicilia| SIC|
 Nord-Est| Bolzano| BOL|
 Nord-Est| Emilia-Romagna| EMR|
 Nord-Est| Trento| TRE|
 Nord-Est| Veneto| VEN|
 Nord-Est| Friuli Venezia Gi...| FVG|
 Nord-Ovest| Lombardia| LOM|
 Nord-Ovest| Liguria| LIG|
 Nord-Ovest| Valle d'Aosta| VDA|
 Nord-Ovest| Piemonte| PIE|
 Sud| Campania| CAM|
 Sud| Abruzzo| ABR|
 Sud| Calabria| CAL|
 Sud| Molise| MOL|
 Sud| Puglia| PUG|
 Sud| Basilicata| BAS|
+------------------+---------------------+-------------+

## Dati Covid Province

In [0]:
dati_covid_province = dati_covid_province.dropna(subset='sigla_provincia') \
                        .drop('stato', 'codice_regione', 'codice_provincia', 'lat', 'long', 'note')
dati_covid_province.show()

+-------------------+---------------------+-----------------------+---------------+-----------+-------------+-------------+-------------+
 data|denominazione_regione|denominazione_provincia|sigla_provincia|totale_casi|codice_nuts_1|codice_nuts_2|codice_nuts_3|
+-------------------+---------------------+-----------------------+---------------+-----------+-------------+-------------+-------------+
2020-02-24 18:00:00| Abruzzo| L'Aquila| AQ| 0| null| null| null|
2020-02-24 18:00:00| Abruzzo| Teramo| TE| 0| null| null| null|
2020-02-24 18:00:00| Abruzzo| Pescara| PE| 0| null| null| null|
2020-02-24 18:00:00| Abruzzo| Chieti| CH| 0| null| null| null|
2020-02-24 18:00:00| Basilicata| Potenza| PZ| 0| null| null| null|
2020-02-24 18:00:00| Basilicata| Matera| MT| 0| null| null| null|
2020-02-24 18:00:00| Calabria| Cosenza| CS| 0| null| null| null|
2020-02-24 18:00:00| Calabria| Catanzaro| CZ| 0| null| null| null|
2020-02-24 18:00:00| Calabria| Reggio di Calabria| RC| 0| null| null| null|
2020-02-24 18:00:00| Calabria| Crotone| KR| 0| null| null| null|
2020-02-24 18:00:00| Calabria| Vibo Valentia| VV| 0| null| null| null|
2020-02-24 18:00:00| Campania| Caserta| CE| 0| null| null| null|
2020-02-24 18:00:00| Campania| Benevento| BN| 0| null| null| null|
2020-02-24 18:00:00| Campania| Napoli| NA| 0| null| null| null|
2020-02-24 18:00:00| Campania| Avellino| AV| 0| null| null| null|
2020-02-24 18:00:00| Campania| Salerno| SA| 0| null| null| null|
2020-02-24 18:00:00| Emilia-Romagna| Piacenza| PC| 0| null| null| null|
2020-02-24 18:00:00| Emilia-Romagna| Parma| PR| 0| null| null| null|
2020-02-24 18:00:00| Emilia-Romagna| Reggio nell'Emilia| RE| 0| null| null| null|
2020-02-24 18:00:00| Emilia-Romagna| Modena| MO| 0| null| null| null|
+-------------------+---------------------+-----------------------+---------------+-----------+-------------+-------------+-------------+
only showing top 20 rows

In [0]:
dati_covid_province.select([count(when(isnull(c), c)).alias(c) for c in dati_covid_province.drop('data').columns]).show()

+---------------------+-----------------------+---------------+-----------+-------------+-------------+-------------+
denominazione_regione|denominazione_provincia|sigla_provincia|totale_casi|codice_nuts_1|codice_nuts_2|codice_nuts_3|
+---------------------+-----------------------+---------------+-----------+-------------+-------------+-------------+
 0| 0| 0| 0| 34882| 34882| 34882|
+---------------------+-----------------------+---------------+-----------+-------------+-------------+-------------+

In [0]:
data = dati_covid_province.groupBy('codice_nuts_1', 'codice_nuts_2', 'codice_nuts_3', 'denominazione_regione', 'denominazione_provincia').count().drop('count').dropna().distinct()

dati_covid_province = dati_covid_province.withColumnRenamed('codice_nuts_1', 'codice_nuts_1_old')
dati_covid_province = dati_covid_province.withColumnRenamed('codice_nuts_2', 'codice_nuts_2_old')
dati_covid_province = dati_covid_province.withColumnRenamed('codice_nuts_3', 'codice_nuts_3_old')

dati_covid_province = dati_covid_province.join(data, on=['denominazione_regione', 'denominazione_provincia'], how='left')
dati_covid_province = dati_covid_province.drop('codice_nuts_1_old', 'codice_nuts_2_old', 'codice_nuts_3_old')
dati_covid_province.show()

+---------------------+-----------------------+-------------------+---------------+-----------+-------------+-------------+-------------+
denominazione_regione|denominazione_provincia| data|sigla_provincia|totale_casi|codice_nuts_1|codice_nuts_2|codice_nuts_3|
+---------------------+-----------------------+-------------------+---------------+-----------+-------------+-------------+-------------+
 Abruzzo| L'Aquila|2020-02-24 18:00:00| AQ| 0| ITF| ITF1| ITF11|
 Abruzzo| Teramo|2020-02-24 18:00:00| TE| 0| ITF| ITF1| ITF12|
 Abruzzo| Pescara|2020-02-24 18:00:00| PE| 0| ITF| ITF1| ITF13|
 Abruzzo| Chieti|2020-02-24 18:00:00| CH| 0| ITF| ITF1| ITF14|
 Basilicata| Potenza|2020-02-24 18:00:00| PZ| 0| ITF| ITF5| ITF51|
 Basilicata| Matera|2020-02-24 18:00:00| MT| 0| ITF| ITF5| ITF52|
 Calabria| Cosenza|2020-02-24 18:00:00| CS| 0| ITF| ITF6| ITF61|
 Calabria| Catanzaro|2020-02-24 18:00:00| CZ| 0| ITF| ITF6| ITF63|
 Calabria| Reggio di Calabria|2020-02-24 18:00:00| RC| 0| ITF| ITF6| ITF65|
 Calabria| Crotone|2020-02-24 18:00:00| KR| 0| ITF| ITF6| ITF62|
 Calabria| Vibo Valentia|2020-02-24 18:00:00| VV| 0| ITF| ITF6| ITF64|
 Campania| Caserta|2020-02-24 18:00:00| CE| 0| ITF| ITF3| ITF31|
 Campania| Benevento|2020-02-24 18:00:00| BN| 0| ITF| ITF3| ITF32|
 Campania| Napoli|2020-02-24 18:00:00| NA| 0| ITF| ITF3| ITF33|
 Campania| Avellino|2020-02-24 18:00:00| AV| 0| ITF| ITF3| ITF34|
 Campania| Salerno|2020-02-24 18:00:00| SA| 0| ITF| ITF3| ITF35|
 Emilia-Romagna| Piacenza|2020-02-24 18:00:00| PC| 0| ITH| ITH5| ITH51|
 Emilia-Romagna| Parma|2020-02-24 18:00:00| PR| 0| ITH| ITH5| ITH52|
 Emilia-Romagna| Reggio nell'Emilia|2020-02-24 18:00:00| RE| 0| ITH| ITH5| ITH53|
 Emilia-Romagna| Modena|2020-02-24 18:00:00| MO| 0| ITH| ITH5| ITH54|
+---------------------+-----------------------+-------------------+---------------+-----------+-------------+-------------+-------------+
only showing top 20 rows

In [0]:
dati_covid_province.select([count(when(isnull(c), c)).alias(c) for c in dati_covid_province.drop('data').columns]).show()

+---------------------+-----------------------+---------------+-----------+-------------+-------------+-------------+
denominazione_regione|denominazione_provincia|sigla_provincia|totale_casi|codice_nuts_1|codice_nuts_2|codice_nuts_3|
+---------------------+-----------------------+---------------+-----------+-------------+-------------+-------------+
 0| 0| 0| 0| 0| 0| 0|
+---------------------+-----------------------+---------------+-----------+-------------+-------------+-------------+

In [0]:
temp = popolazione_regioni.withColumnRenamed('denominazione_regione', 'denominazione_regione_1')

diff = temp.groupBy('denominazione_regione_1').count().drop('count').subtract(dati_covid_province.groupBy('denominazione_regione').count().drop('count'))

# TODO: NON ABBIAMO RISOLTO STO PROBLEMA
# diff = diff.withColumnRenamed('denominazione_regione', 'denominazione_regione_1')
filter(temp.denominazione_regione_1 == diff.denominazione_regione_1)
temp.groupBy('codice_nuts_2', 'denominazione_regione_1').count().drop('count').join(dati_covid_province.groupBy('codice_nuts_2', 'denominazione_regione').count().drop('count'), how='outer', on='codice_nuts_2').show(100)

+-------------+-----------------------+---------------------+
codice_nuts_2|denominazione_regione_1|denominazione_regione|
+-------------+-----------------------+---------------------+
 ITC1| Piemonte| Piemonte|
 ITC2| Valle d'Aosta| Valle d'Aosta|
 ITC3| Liguria| Liguria|
 ITC4| Lombardia| Lombardia|
 ITF1| Abruzzo| Abruzzo|
 ITF2| Molise| Molise|
 ITF3| Campania| Campania|
 ITF4| Puglia| Puglia|
 ITF5| Basilicata| Basilicata|
 ITF6| Calabria| Calabria|
 ITG1| Sicilia| Sicilia|
 ITG2| Sardegna| Sardegna|
 ITH1| Bolzano| P.A. Bolzano|
 ITH2| Trento| P.A. Trento|
 ITH3| Veneto| Veneto|
 ITH4| Friuli Venezia Gi...| Friuli Venezia Gi...|
 ITH5| Emilia-Romagna| Emilia-Romagna|
 ITI1| Toscana| Toscana|
 ITI2| Umbria| Umbria|
 ITI3| Marche| Marche|
 ITI4| Lazio| Lazio|
+-------------+-----------------------+---------------------+

In [0]:
diff.show()

+-----------------------+
denominazione_regione_1|
+-----------------------+
 Bolzano|
 Trento|
+-----------------------+

In [0]:
# dati_covid_province = dati_covid_province.replace('P.A. Trento', value='Trento', subset='denominazione_regione')
# dati_covid_province = dati_covid_province.replace('P.A. Bolzano', value='Bolzano', subset='denominazione_regione')

num_regioni = dati_covid_province.groupBy('denominazione_regione') \
                                    .count() \
                                    .count()

print('Il numero di regioni è:', num_regioni)

dati_covid_province.groupBy('denominazione_regione') \
                    .count() \
                    .drop('count') \
                    .show(num_regioni)

In [0]:
num_province = dati_covid_province.groupBy('denominazione_provincia') \
                                    .count() \
                                    .count()

print('Il numero di province è:', num_province)

dati_covid_province.groupBy('denominazione_provincia', 'sigla_provincia') \
                      .count() \
                      .drop('count') \
                      .orderBy('denominazione_provincia') \
                      .show(num_province)

## Dati Covid Regioni

In [0]:
dati_covid_regioni = dati_covid_regioni.drop('stato', 'lat', 'long', 'note_test', 'note_casi')

dati_covid_regioni.show()

# dati_covid_regioni.filter(dati_covid_regioni.casi_da_screening.isNotNull()).show()

In [0]:
dati_covid_regioni.select([count(when(isnull(c), c)).alias(c) for c in dati_covid_regioni.drop('data').columns]).show()